In [2]:
import torch
from torch import nn
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

In [9]:
conv.weight.shape

torch.Size([128, 256, 3, 3, 3])

In [3]:
x = torch.rand(1, 256, 16, 16, 16)
conv = nn.Conv3d(256, 128, 3)
with torch.inference_mode():
    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        conv(x)

STAGE:2023-10-28 23:12:34 16136:16136 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-10-28 23:12:34 16136:16136 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-10-28 23:12:34 16136:16136 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [7]:
print(prof.key_averages().table())

----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                        Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                aten::conv3d         4.76%       2.282ms       100.00%      47.938ms      47.938ms             1  
           aten::convolution         0.07%      32.000us        95.24%      45.656ms      45.656ms             1  
          aten::_convolution         2.39%       1.145ms        95.17%      45.624ms      45.624ms             1  
    aten::mkldnn_convolution        92.17%      44.184ms        92.78%      44.479ms      44.479ms             1  
                 aten::empty         0.05%      26.000us         0.05%      26.000us      13.000us             2  
           aten::as_strided_         0.02%      11.000us         0.02%      11.0